# Final Project
### Import precomp libs


In [1]:
#import the spacy english model
# "python -m spacy download en"   to download english models
import spacy
nlp = spacy.load('en') # this should take some time like 10s to load
import numpy as np
import scipy as sp
import pandas as pd


In [4]:
# PROJECT structure: data directory with the two csvs of train,test and the ipython notbook in its own directory
# loading the data
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
print train.shape, test.shape

(404290, 6) (2345796, 3)


In [5]:
train.head(5)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [ ]:
#this procedure will be mapped to each row
def eval(row):
    q1_txt = nlp(row['question1'])
    q2_txt = nlp(row['question2'])

    # feature: similarity measure from built-in spacy
    #word net similarity? or hamming distance of the strings?
    sy_sim = q1_txt.similarity(q2_txt)

    #feature: 
    return sy_sim
    
#unit test(s)
test = pd.dataFrame()
print eval()